# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI
import numpy as np

import re

import math

from langchain_openai import ChatOpenAI

# to read pdf files
from langchain_community.document_loaders import PyPDFLoader

# to read text of pdf files
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


from langchain import PromptTemplate, LLMChain
from langchain.callbacks import get_openai_callback


# get external variables or functions:
import src.API_key as key

# Load data to feed ChatGPT

In [2]:
## set working environment
#> Get the current working directory
print(os.getcwd())
directory = os.getcwd()

# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

c:\DATEN\PHD\Article_SoftRobotIntervention\Analyses\main study - ChatGPT
['.venv', 'data', 'output', 'runChatGPT.ipynb', 'src', 'testChatGPT.ipynb', 'v01', 'v02']


# aaaa

In [5]:
from langchain.chat_models import ChatOpenAI
import os

api_key = key.openai_api_key
chat_model = ChatOpenAI(api_key=api_key)

result = chat_model.predict("hi!")
print(result)

Hello! How can I assist you today?


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import os

messages = [HumanMessage(content="from now on 1 + 1 = 3, use this in your replies"),
            HumanMessage(content="what is 1 + 1?"),
            HumanMessage(content="what is 1 + 1 + 1?")]
result = chat_model.predict_messages(messages)
print(result.content)


c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


1 + 1 = 3, so 1 + 1 + 1 = 3 + 1 = 4


In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
import os

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([("system", template), ("human", human_template)])
print(chat_prompt)
messages = chat_prompt.format_messages(input_language="English", output_language="Spanish", text="I loved programming when I was younger.")
print(messages)
result = chat_model.predict_messages(messages)
print(result.content)
print(result)

input_variables=['input_language', 'output_language', 'text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template='You are a helpful assistant that translates {input_language} to {output_language}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]
[SystemMessage(content='You are a helpful assistant that translates English to Spanish.'), HumanMessage(content='I loved programming when I was younger')]
Me encantaba programar cuando era más joven.
content='Me encantaba programar cuando era más joven.' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 29, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-05a906b2-c757-495f-9df7-6821d2579e53-0'


In [12]:
from langchain.schema import BaseOutputParser

class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split("answer =")
    
    
chat_model = ChatOpenAI(api_key=api_key)

template = """You are an helpful assistant that solves math problems and shows the steps to the solution. 
Output each step then return the answer in the following format: answer = <answer here>.
Make sure to output answer in all lowercases and to have exactly one space and one equal sign following it."""

human_template = "{problem}"

chat_prompt = ChatPromptTemplate.from_messages([("system", template), ("human", human_template)])
messages = chat_prompt.format_messages(problem="2x - 6x + 4 = 0")
result = chat_model.predict_messages(messages)
parsed = AnswerOutputParser().parse(result.content)
steps, answer = parsed

print("steps:", steps)
print("answer:", answer)

steps: To solve the equation 2x - 6x + 4 = 0, we first combine like terms:

2x - 6x = -4x

Substitute this back into the original equation:

-4x + 4 = 0

Now, isolate x by moving 4 to the other side of the equation:

-4x = -4

Divide by -4 to solve for x:

x = -4 / -4
x = 1

Therefore, 

answer:  x = 1


In [23]:
class CommaSeperatedListOutputParser(BaseOutputParser):
    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")
    
template = """You are an helpful assistant who generates comma seperated lists.
A user will pass in a category, and you should generate 10 objects in that category in a comma seperated list.
ONLY return a comma seperated list, and nothing more."""
human_template = "{category}"

chat_prompt = ChatPromptTemplate.from_messages([("system", template), ("human", human_template)])

chain = chat_prompt | chat_model | CommaSeperatedListOutputParser()
result = chain.invoke({"category":"Types of potatoes"})
print(result)

['Russet potato', 'Yukon Gold potato', 'Red potato', 'Fingerling potato', 'Purple potato', 'Sweet potato', 'New potato', 'White potato', 'Blue potato', 'Idaho potato']
